<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Generate Vector Embeddings using Huggingface model and Script Table Operator(STO)
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Hugging Face is a French-American company based in New York City that develops computation tools for building applications using machine learning. They are known for their <b>Transformers Library</b> which provides open-source implementations of transformer models for text, image, video, audio tasks including time-series. These models include well-known architectures like BERT and GPT. The library is compatible with PyTorch, TensorFlow, and JAX deep learning libraries. <br>
    Deep Learning Models in HuggingFace are pretrained by users/open source outfits/companies on various types of data – NLP, Audio, Images, Videos etc. Most popular tool of choice by users is PyTorch (open source python library) which helps create a Deep Learning model from scratch or take an existing model, retrain/fine-tune (Transfer Learning) on new set of data to be published in HF. Models can be inferenced with CPUs and GPUs with slight performance improvement for smaller models.<br>
</p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>  
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As many Hugging Face models are availble in ONNX Runtime, we can load them using the BYOM feature of Vantage and run them in Vantage. Because of Graph Optimizations on ONNX Runtime, there are proven benchmarks that show that inference with ONNX Runtime will be 20% faster than a native PyTorch model on a CPU. Vantage Parallelism on top of boosted ONNX Runtime inference can turn a Vantage system as effective as inference on GPUs. If we have a Vantage box with 72 AMPs, assuming the table is perfectly distributed, it will closely match the performance of a dedicated GPU and data never moves across the network saving time and I/O operations. As parallelism increases with number of AMPs, the model inference will complete faster in Teradata Vantage with the same amount of text data vs a GPU. We can of course quantize the model (change float8 weights to int8/int4) for inference on CPU to go even faster with some tradeoff with accuracy. However, If Model size goes up GPU advantage will widen – example LLM like LLama2 and costs will be disproportionate with GPU but for smaller models we can get comparable performance. 
</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    This notebook demonstrates the ability to use Huggingface model in Vantage Script table operator(STO) for an On Prem Enterprise edition of Vantage.
</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
!pip install tdstone2 --upgrade

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The kernel must be restarted to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

from teradataml import (
    create_context,
    execute_sql, 
    copy_to_sql, 
    DataFrame,
    remove_context,
    FillRowId)
import tdstone2

import json

# Modify the following to match the specific client environment settings
display.max_rows = 5


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Vector_Embeddings_STO_Python.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Getting Data for This Demo</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will generate the required data. The data we are creating is categorized by typical software issues and some questions that are typically asked. To simplify the process we will insert the data into a python dictionary, load it into pandas dataframe, and than copy the dataframeinto a table in Vantage.</p> 

In [ ]:
import pandas as pd
import numpy as np

# Define the three types of software problems and corresponding questions
problems_data = {
    "Problem_Type": ["Installation Issue", "Performance Issue", "Functionality Issue"],
    "User_Question": [
        [
            "Why can't I install the software on my machine?",
            "What do I do if the installer keeps crashing?",
            "How do I resolve dependency errors during installation?",
            "Why is my antivirus blocking the software installation?"
        ],
        [
            "Why is the software running so slowly?",
            "How do I fix memory issues causing the software to crash?",
            "What can I do if the software takes too long to load?",
            "Why is the CPU usage so high when using the software?"
        ],
        [
            "Why is the 'Save' button not working?",
            "How do I troubleshoot errors when trying to export data?",
            "Why does the software keep freezing when I try to open certain files?",
            "What should I do if features are missing after an update?"
        ]
    ]
}

# Convert the dictionary into a pandas DataFrame
df = pd.DataFrame(problems_data)

# Expanding the dataframe so each row corresponds to one question
expanded_rows = []

for index, row in df.iterrows():
    problem_type = row["Problem_Type"]
    questions = row["User_Question"]
    for question in questions:
        expanded_rows.append({"Problem_Type": problem_type, "User_Question": question})

# Create a new DataFrame with the expanded rows
df = pd.DataFrame(expanded_rows)
df['id'] = df.index
df = df[['id','Problem_Type','User_Question']]
df

In [ ]:
copy_to_sql(df,  table_name = 'questions', if_exists = 'replace')
dataset = DataFrame('questions')
dataset

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Data Distribution</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will check the number of AMPs available on our system and than based on that we will distribute the data equally on all amps to utilize the complete power of the sytem available to us.</p> 

In [ ]:
nb_amps = execute_sql('SEL HASHAMP()').fetchall()[0][0]+1
nb_amps

In [ ]:
from tdstone2.data_distribution import InverseHash, EquallyDistribute
from tdstone2.dataset_generation import gen_query

In [ ]:
df = gen_query(dataset[['User_Question']],n=1)[['User_Question']] # Generate a single partition
df = gen_query(df, n=nb_amps, replication_column = 'Partition_ID')
df

In [ ]:
df = EquallyDistribute(df)

In [ ]:
df = df.assign(Problem_Type = df.Partition_ID)
df = FillRowId(data=df,
                    row_id_column='Id'
                   ).result
df[['Id','Problem_Type','User_Question']].to_sql(
    table_name='questions_large',
    primary_index = 'Id',
    if_exists = 'replace'
)
dataset_large = DataFrame('questions_large')
dataset_large

In [ ]:
from tdstone2.data_distribution import PlotDistribution
PlotDistribution('demo_user', 'questions_large', partition = 'Problem_Type')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Installing the files in Vantage</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will Install tdstone2 framework. First step will be to clean up the schema and than setup the framework for sto and than install the required files in Vantage.</p> 

In [ ]:
from tdstone2.utils import cleanup
cleanup(schema='demo_user')
         

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the tdstone2 package to install the generic python files that enables the ML training, scoring, feature engineering and vector embedding computations. These files are installed once, and enable user-friendly interactions with the platform. The vector embeddings will be displayed in the following cells.</p> 

In [ ]:
from tdstone2.tdstone import TDStone
sto = TDStone(schema_name = 'demo_user', SEARCHUIFDBPATH = 'demo_user')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will install the necessary libraries into the sto environment of Vantage. PushFile installs the py file, including the py file that implements the vector embedding.</p> 

In [ ]:
sto.PushFile()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will than install the required models into Vantage. Here, we are using the <code>BAAI/bge-small-en-v1.5</code> model.</p> 

In [ ]:
from tdstone2.tdsgenai import install_model_in_vantage_from_name
install_model_in_vantage_from_name(sequence_length=256, model_name = 'BAAI/bge-small-en-v1.5', model_task = 'feature-extraction', replace = False)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will check the install models using the list function and as we can see the 2 models we installed above are now available in the Vantage environment.</p> 

In [ ]:
from tdstone2.tdsgenai import list_installed_files
list_installed_files()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Compute Vector Embeddings</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will than compute the embeddings using the compute function which will use the model and the dataset and than generate the embeddings. The steps involved in the process will be displayed as a part of the output.  compute_vector_embedding function sets up and executes a script for the given model and dataset, ensuring that the text column is VARCHAR
and the model exists. Finally, create a pivot view of the results.</p> 

In [ ]:
from tdstone2.tdsgenai import compute_vector_embedding

In [ ]:
res = compute_vector_embedding(
    # choose your language model
    model              = 'tdstone2_emb_256_BAAI_bge-small-en-v1_5.zip',
    # the description of the dataset
    dataset            = dataset,           # the teradata dataframeS
    text_column        = 'User_Question',   # the column containing the text
    hash_columns       = ['id'],            # the hash column to distribute the computations
    accumulate_columns = ['Problem_Type'],  # the columns we want to keep in the output results
    # the output table
    schema_name        = 'demo_user', # the database 
    table_name         = 'embeddings'       # the table name (actually a view on a result table)
)

In [ ]:
res

In [ ]:
computation_time = res.groupby('jobid').agg({'process_time':['sum','max','count','median'], 'elapsed_time':['sum','max','count','median']})
computation_time

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:##00233C'><b>Work Tables</b></p>

In [ ]:
tables = ['questions', 'questions_large','embeddings']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass
  
   

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>